In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt 

### Momentum Trading Strategy 

In [ ]:
#Arrnage Data
btc = pdr.get_data_yahoo('BTC-USD')
day = np.arange(1, len(btc) + 1)
btc['day'] = day
btc.drop(columns=['Adj Close', 'Volume'], inplace = True)
btc = btc[['day', 'Open', 'High', 'Low', 'Close']]
btc.head()

In [ ]:
# add moving average 
btc['9-day'] = btc['Close'].rolling(9).mean().shift()
btc['21-day'] = btc['Close'].rolling(21).mean().shift()
btc[19:25]

In [ ]:
# add Signal column
btc ['signal'] = np.where(btc['9-day'] > btc['21-day'], 1, 0)
btc['signal'] = np.where(btc['9-day'] < btc['21-day'], -1, btc['signal'])
btc.dropna(inplace=True)

In [ ]:
#calculate buy and hold return vs strategy return 
btc['return'] = np.log(btc['Close']).diff()
btc['strategy_return'] = btc['signal'] * btc['return']
btc['entry'] = btc.signal.diff()
btc.head()

In [ ]:
#plot trades on time series

plt.rcParams['figure.figsize'] = 12, 6 
plt.grid(True, alpha = .3)
plt.plot(btc.iloc[-365:]['Close'], label = 'BTC')
plt.plot(btc.iloc[-365:]['9-day'], label = '9-day')
plt.plot(btc.iloc[-365:]['21-day'], label = '21-day')
plt.plot(btc[-365:].loc[btc.entry == 2].index, btc[-365:]['9-day'][btc.entry == 2], '^', color = 'g', markersize = 12)
plt.plot(btc[-365:].loc[btc.entry == -2].index, btc[-365:]['21-day'][btc.entry == -2], 'v', color = 'r', markersize = 12)
plt.legend(loc=2);


In [ ]:
plt.plot(np.exp(btc.iloc[-365:]['return']).cumprod(), label='Buy/Hold')
plt.plot(np.exp(btc.iloc[-365:]['strategy_return']).cumprod(), label='Strategy')
plt.legend(loc=2)
plt.grid(True, alpha=.3)

In [ ]:
np.exp(btc.iloc[-365:]['return']).cumprod()[-1] -1

In [ ]:
np.exp(btc.iloc[-365:]['strategy_return']).cumprod()[-1] -1